# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
from time import time

import keras.backend as K
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, MaxPooling1D, Concatenate
from tensorflow.keras.layers import Dense, Conv1D, Dropout, BatchNormalization, LeakyReLU, ELU
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Data loading

In [3]:
n_tips = ['674', '489', '87']

In [4]:
pickle_base = '/workspace/coniferas(1)/data_inference/pickles/simulations_no_fossil/dataset_'
data = dict()
for i in n_tips:
    with open(pickle_base + i + "_10k.pkl", 'rb') as f:
        data[i] = pickle.load(f)

# Model definition

In [5]:
def create_nn(n_out, n_tips, clas=False, div_scenario = None):
    input_data = Input(shape=(n_tips, 1))

    final_filters = 128
    x = Conv1D(16, kernel_size=3, padding='same')(input_data)
    x = ELU()(x)
    x = Conv1D(16, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = MaxPooling1D(pool_size=2)(x)
    
    if n_tips > 256:
        final_filters = 64
        x = Conv1D(32, kernel_size=3, padding='same')(x)
        x = ELU()(x)
        x = Conv1D(32, kernel_size=3, padding='same')(x)
        x = ELU()(x)
        x = MaxPooling1D(pool_size=2)(x)
    
        if n_tips > 512:
            final_filters = 128
            x = Conv1D(64, kernel_size=3, padding='same')(x)
            x = ELU()(x)
            x = Conv1D(64, kernel_size=3, padding='same')(x)
            x = ELU()(x)
            x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(final_filters, kernel_size=3, padding='same')(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(32)(x)
    x = ELU()(x)
    x = Dropout(0.3)(x)
    
    x = Dense(n_out)(x)
    if clas:
        output_class = Softmax()(x)
        
    else:
        if div_scenario != "SAT":
            out_list = []

            for i in range(n_out):
                y = Dense(32)(x)
                y = ELU()(y)
                y = Dropout(0.3)(y)

                y = Dense(1)(y)
                y = LeakyReLU(alpha=10)(y)
                out_list.append(y)

            output_class = Concatenate()(out_list)
            
        elif div_scenario == "WW":
            out_list = []

            for i in range(n_out):
                y = Dense(32)(x)
                y = ELU()(y)
                y = Dropout(0.3)(y)

                y = Dense(1)(y)
                y = Linear(y)
                out_list.append(y)

            output_class = Concatenate()(out_list)                
            
        else:
            x = Dense(32)(x)
            x = ELU()(x)
            x = Dropout(0.3)(x)
            x = Dense(1)(x)
            output_class = LeakyReLU(alpha=10)(x)
            
    return Model(input_data, output_class)

# Training

In [6]:
callback = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
out_folder_path = "/workspace/coniferas(1)/data_inference/models/"
os.makedirs(out_folder_path + 'class/', exist_ok=True)
os.makedirs(out_folder_path + 'reg/', exist_ok=True)

## Classification training

In [7]:
for i in n_tips:
    print("\nClasification training", i, 'tips')

    nn_model = create_nn(len(data[i]['y_class_train'][0]),
                         int(i), clas=True)
    nn_model.compile(loss="categorical_crossentropy",
                     optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    start = time()
    history = nn_model.fit(data[i]['X_train'], data[i]['y_class_train'],
                           batch_size=128, epochs=1000, validation_split=0.1,
                           callbacks=[callback], verbose=0)
    elapsed_time = time()-start
    print('Elapsed time', elapsed_time)

    save_path = out_folder_path + 'class/' + i + "_classification_"

    nn_model.save(save_path + "model.keras")
    with open(save_path + "history.pkl", 'wb') as f:
            pickle.dump(history.history, f)
    with open(save_path + "model_data.pkl", 'wb') as f:
            pickle.dump([nn_model.count_params(), elapsed_time], f)


Clasification training 674 tips
Elapsed time 913.5207743644714

Clasification training 489 tips
Elapsed time 1722.0513048171997

Clasification training 87 tips
Elapsed time 1402.9934368133545


## Regression training

In [11]:
for i in n_tips:
    for label in np.unique(data[i]['div_info_train']):
        div_scenario = label.split('/')[1].split('_')[0]
        
        for data_norm in ['_norm', '']:
            # Get regression values of the corresponding scenario
            X_train = data[i]['X_train'][data[i]['div_info_train'] == label]
            y_reg_train = data[i]['y_reg' + data_norm + '_train'][data[i]['div_info_train'] == label]
            y_reg_train = [np.array(elem) for elem in y_reg_train]
            
            print("\nRegression training", i, 'tips,', div_scenario, 'model,', data_norm, 'norm')
            nn_model = create_nn(len(y_reg_train[0]),
                                 int(i), div_scenario=div_scenario)
            nn_model.compile(loss="mae", optimizer=Adam(learning_rate=0.001),
                             metrics=['mse'])

            start = time()
            history = nn_model.fit(np.expand_dims(X_train, axis=2),
                                   np.expand_dims(y_reg_train, axis=2),
                                   batch_size=128, epochs=1000, validation_split=0.1,
                                   callbacks=[callback], verbose=0)
            elapsed_time = time()-start
            print('Elapsed time', elapsed_time)

            save_path = out_folder_path + 'reg/' + div_scenario + '/'
            os.makedirs(save_path, exist_ok=True)
            save_path +=  i + "_regression" + data_norm + '_'

            nn_model.save(save_path + "model.keras")
            with open(save_path + "history.pkl", 'wb') as f:
                    pickle.dump(history.history, f)
            with open(save_path + "model_data.pkl", 'wb') as f:
                    pickle.dump([nn_model.count_params(), elapsed_time], f)


Regression training 674 tips, BD model, _norm norm
Elapsed time 72.86232209205627

Regression training 674 tips, BD model,  norm
Elapsed time 65.01599049568176

Regression training 674 tips, HE model, _norm norm
Elapsed time 82.6969530582428

Regression training 674 tips, HE model,  norm
Elapsed time 58.69716143608093

Regression training 674 tips, ME model, _norm norm
Elapsed time 84.64487767219543

Regression training 674 tips, ME model,  norm
Elapsed time 90.6526095867157

Regression training 674 tips, SAT model, _norm norm
Elapsed time 71.70607733726501

Regression training 674 tips, SAT model,  norm
Elapsed time 53.84978413581848

Regression training 674 tips, SR model, _norm norm
Elapsed time 81.4151291847229

Regression training 674 tips, SR model,  norm
Elapsed time 96.70000576972961

Regression training 674 tips, WW model, _norm norm
Elapsed time 94.1911301612854

Regression training 674 tips, WW model,  norm
Elapsed time 121.00370073318481

Regression training 489 tips, BD m